## Corporación Favorita Sales Forecasting - Baseline

In [1]:
import itertools

In [2]:
import pandas as pd
import numpy as np

In [4]:
PATH='data/grocery/'

In [4]:
df_train = pd.read_csv(f'{PATH}train.csv', usecols=[1,2,3,4,5], 
                       dtype={'onpromotion': str}, 
                       converters={'unit_sales': lambda u: float(u) if float(u) > 0
                                  else 0},
                                skiprows=range(1, 124035460))

### Log transform

In [5]:
df_train["unit_sales"] =  df_train["unit_sales"].apply(np.log1p)

Fill gaps in dates

In [6]:
u_dates = df_train.date.unique()
u_stores = df_train.store_nbr.unique()
u_items = df_train.item_nbr.unique()
df_train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)

In [7]:
df_train = df_train.reindex(
    pd.MultiIndex.from_product((u_dates, u_stores, u_items),
    names=["date", "store_nbr", "item_nbr"]))

Fill NAs

In [8]:
df_train.loc[:, "unit_sales"].fillna(0, inplace=True)

In [9]:
df_train.unit_sales

date        store_nbr  item_nbr
2017-08-02  1          96995       0.693147
                       103520      0.693147
                       103665      1.098612
                       105574      2.197225
                       105575      2.197225
                       105857      1.791759
                       106716      0.693147
                       108696      1.386294
                       108701      1.098612
                       108786      2.197225
                       108797      1.791759
                       108862      1.386294
                       111223      2.708050
                       111397      1.098612
                       112830      1.791759
                       114778      1.386294
                       114790      1.945910
                       114800      1.386294
                       115611      2.079442
                       115693      1.609438
                       115720      0.693147
                       115891      1.609438


Assume Missing entries imply no promotion

In [10]:
df_train.loc[:, "onpromotion"].fillna('Flase', inplace=True)

Calculate means

In [11]:
df_train = df_train.groupby(
    ['item_nbr', 'store_nbr', 'onpromotion']
)['unit_sales'].mean().to_frame('unit_sales')

Inverse transform

In [12]:
df_train["unit_sales"] = df_train["unit_sales"].apply(np.expm1)

In [14]:
pd.read_csv(f'{PATH}test.csv', usecols=[0,2,3,4], dtype={'onpromotion': str}
           ).set_index(['item_nbr', 'store_nbr', 'onpromotion']
                      ).join(df_train, how='left'
                            ).fillna(0).to_csv(f'{PATH}mean1.csv.gz', float_format='%.2f',index=None,
                                              compression="gzip")

In [6]:
mf=pd.read_csv(f'{PATH}mean1.csv')

In [16]:
mf.tail()

,id,unit_sales
3370459,128024887,0.0
3370460,128235541,0.0
3370461,128446195,0.0
3370462,128656849,0.0
3370463,128867503,0.0


In [9]:
mf.describe()

,id,unit_sales
count,3.370464e+06,3.370464e+06
mean,1.271823e+08,3.619123e+00
std,9.729693e+05,1.072351e+01
min,1.254970e+08,0.000000e+00
25%,1.263397e+08,0.000000e+00
50%,1.271823e+08,1.670000e+00
75%,1.280249e+08,3.560000e+00
max,1.288675e+08,2.182000e+03


In [14]:
from IPython.display import FileLink